In [38]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [39]:

# Path ke driver dan direktori profil Chrome
driver_path = '/path/to/chromedriver'  # Ganti dengan path driver Anda
# profile_path = r'C:\\Users\\LENOVO\\AppData\\Local\\Google\\Chrome\\User Data\\Default'  # Path profil Chrome yang sudah login
# profile_path = 'C:\\Users\\LENOVO\\AppData\\Local\\Google\\Chrome\\User Data\\Default'  # Ganti path ke profil Chrome Anda
profile_path = r'C:\Users\LENOVO\AppData\Local\Google\Chrome\User Data\Default'  # Ganti sesuai path profil Chrome Anda


In [40]:
try:
    response = requests.get("https://www.youtube.com")
    if response.status_code == 200:
        print("Koneksi ke YouTube berhasil.")
    else:
        print("Koneksi ke YouTube tidak berhasil.")
except requests.ConnectionError as e:
    print("Gagal menghubungi YouTube:", e)

Koneksi ke YouTube berhasil.


In [41]:

def initialize_driver_with_profile(profile_path):
    """Inisialisasi driver dengan profil Chrome yang sudah ada."""
    chrome_options = Options()
    chrome_options.add_argument(f"user-data-dir={profile_path}")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--enable-logging")
    chrome_options.add_argument("--v=1")  # Level logging dasar
    # service = Service(driver_path)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def get_all_youtube_comments(driver, video_url, max_comments=None):
    """Mengambil semua komentar dari video YouTube dengan scroll tanpa batas."""
    driver.get(video_url)
    time.sleep(3)  # Tunggu halaman dimuat

    comments = []
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll ke bawah untuk memuat komentar baru
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)  # Jeda untuk memuat komentar baru

        # Ambil elemen komentar
        comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        for element in comment_elements[len(comments):]:  # Mulai dari komentar baru
            comment_text = element.text
            if comment_text:
                comments.append(comment_text)
                # Batas maksimal komentar jika disetel
                if max_comments and len(comments) >= max_comments:
                    return comments

        # Cek tinggi baru halaman untuk melihat apakah ada yang baru dimuat
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:  # Berhenti jika tidak ada lagi yang dimuat
            break
        last_height = new_height

    return comments

def save_to_csv(comments, filename="youtube_comments.csv"):
    """Menyimpan komentar ke file CSV."""
    df = pd.DataFrame(comments, columns=["Comment"])
    df.to_csv(filename, index=False)
    print(f"Komentar berhasil disimpan di {filename}")


In [42]:
# Inisialisasi WebDriver dengan profil Chrome
driver = initialize_driver_with_profile(profile_path)
# URL video YouTube dan jumlah komentar maksimal yang diinginkan (bisa None untuk semua)
# video_url = "https://www.youtube.com/watch?v=i6IOiUi6IYY"  # Ganti dengan ID video yang diinginkan
# URL video YouTube dan jumlah komentar maksimal yang diinginkan
# idyoutube="6IOiUi6IYY"
video_url = f"https://www.youtube.com/watch?v=i6IOiUi6IYY"  # Menggunakan idyoutube sebagai variabel

comments = get_all_youtube_comments(driver, video_url, max_comments=None)  # None untuk scroll tanpa batas
# save_to_csv(comments)
# Simpan komentar dengan nama file yang mengandung ID video
filename = f"youtube_comments_i6IOiUi6IYY.csv"
save_to_csv(comments, filename)

# Tutup driver setelah selesai
driver.quit()


Komentar berhasil disimpan di youtube_comments_i6IOiUi6IYY.csv
